In [1]:
from datetime import datetime, date

import pandas as pd
import numpy as np
import datetime 
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
import os
import re
import time
from sklearn.model_selection import train_test_split

# 忽略警告訊息
import warnings
warnings.filterwarnings('ignore')

# 推薦名單需要的套件
import turicreate as tc

#  graphlab 安裝有點問題
# import graphlab

## 安裝教學

安裝 graphlab 步驟 https://www.jianshu.com/p/65656ad89dc6

安裝 graphlab要註冊帳號拿金鑰 5/13已註冊可提供一年權限
輸入以下代碼：

pip install --upgrade --no-cache-dir https://get.graphlab.com/GraphLab-Create/2.1/a942512005@gmail.com/8E42-3F71-D3DE-D16C-0D09-D9AA-B5B5-1CCD/GraphLab-Create-License.tar.gz

In [5]:
al2['qty'] = 0 # 先透過這種方式新增好欄位做準備，下面執行效能會好很多

for idx, row in al2.iterrows():  # 請務必記得加上idx，不然跑回圈的item會變成(idx, row)
    market_qty = row['market_qty']
    if market_qty < 2:
        qty = 1
    elif market_qty < 4:
        qty = 2
    elif market_qty < 8:
        qty = 3
    elif market_qty < 15:
        qty = 4
    else: 
        qty = 5
    al2.loc[idx, 'qty'] = qty   #r在for迴圈裡面   
al2.head()    

,member_id,細分類,market_qty,qty
0,5,小吃,1,1
1,7,肉乾,6,3
2,12,小吃,6,3
3,12,肉乾,10,4
4,35,小吃,1,1


In [6]:
alo = pd.DataFrame(al1.qty.value_counts()).reset_index()
alo.columns= ['qty','count']

alo[['count']] = alo[['count']].astype('float64')
alo['比例']= np.round((alo['count'] / len(al1.member_id)),4)               
print(alo.sort_values('qty',ascending = True))

al4 = al1.copy() # 保留原本的 後續使用
al2 = al1.drop(columns=['market_qty'], axis =1 ) #用此跑推薦

   qty    count      比例
0    1  35266.0  0.5250
1    2  11040.0  0.1644
2    3  10347.0  0.1540
3    4   5329.0  0.0793
4    5   5187.0  0.0772


In [7]:
old_most = al2.groupby(['member_id']).max().reset_index() #會員曾經買最多的商品 最後推薦會併
old_most = old_most.rename(columns = {'細分類' : '商品'})

print(len(al2.member_id.unique()),len(al2))
al2 = al2.rename(columns = {'細分類' : '商品'})
al2.head()

(64509, 67169)


,member_id,商品,qty
0,5,小吃,1
1,7,肉乾,3
2,12,小吃,3
3,12,肉乾,4
4,35,小吃,1


###  推薦方法 

In [39]:
al2 = tc.SFrame(al2)
# Create train and test set
train_data, test_data = tc.recommender.util.random_split_by_user(al2,user_id = 'member_id',
                                                item_id = '商品', item_test_proportion=0.3, random_seed=221)
# from sklearn.model_selection import train_test_split
# train_data , test_data = train_test_split( al2 , test_size=0.3)
# train_data = graphlab.SFrame(train_data)
# test_data = graphlab.SFrame(test_data)

### 流行度

In [40]:
# Simple popularity-based recommender
popularity_model = tc.popularity_recommender.create(train_data,user_id = 'member_id'
                                    ,item_id='商品' ,target='qty')

Preparing data set.

Data has 66834 observations with 64191 users and 16 items.

Data prepared in: 0.175466s

66834 observations to process; with 16 unique items.

In [41]:
users = al2['member_id'].unique()                                          
# Use the popularity model to make some predictions
# popularity_model.recommend(users=[users[0...66343]])

popularity_model.recommend(users=[users[6]])

member_id,商品,score,rank
5288,肉乾,2.83762032507,1
5288,麵包,2.81338742394,2
5288,餅乾,2.40307461567,3
5288,小吃,2.37938381413,4
5288,蛋捲,2.09652509653,5
5288,牛肉海鮮,1.85294117647,6
5288,包子,1.73698630137,7
5288,魚,1.53472882969,8
5288,素食小吃,1.38095238095,9
5288,蛋糕,1.33768844221,10


In [42]:
# 基於流行度的模型得到推薦結果你會發現，不管你要推薦給那一個使用者，結果都會是一樣的
popularity_recomm = popularity_model.recommend(users=range(1,10),k=3)

popularity_recomm.print_rows(num_rows=27)

+-----------+------+---------------+------+
| member_id | 商品 |     score     | rank |
+-----------+------+---------------+------+
|     1     | 肉乾 | 2.83762032507 |  1   |
|     1     | 麵包 | 2.81338742394 |  2   |
|     1     | 餅乾 | 2.40307461567 |  3   |
|     2     | 肉乾 | 2.83762032507 |  1   |
|     2     | 麵包 | 2.81338742394 |  2   |
|     2     | 餅乾 | 2.40307461567 |  3   |
|     3     | 肉乾 | 2.83762032507 |  1   |
|     3     | 麵包 | 2.81338742394 |  2   |
|     3     | 餅乾 | 2.40307461567 |  3   |
|     4     | 肉乾 | 2.83762032507 |  1   |
|     4     | 麵包 | 2.81338742394 |  2   |
|     4     | 餅乾 | 2.40307461567 |  3   |
|     5     | 肉乾 | 2.83762032507 |  1   |
|     5     | 麵包 | 2.81338742394 |  2   |
|     5     | 餅乾 | 2.40307461567 |  3   |
|     6     | 肉乾 | 2.83762032507 |  1   |
|     6     | 麵包 | 2.81338742394 |  2   |
|     6     | 餅乾 | 2.40307461567 |  3   |
|     7     | 麵包 | 2.81338742394 |  1   |
|     7     | 餅乾 | 2.40307461567 |  2   |
|     7     | 小吃 | 2.379383814

In [43]:
popularity_recomm

member_id,商品,score,rank
1,肉乾,2.83762032507,1
1,麵包,2.81338742394,2
1,餅乾,2.40307461567,3
2,肉乾,2.83762032507,1
2,麵包,2.81338742394,2
2,餅乾,2.40307461567,3
3,肉乾,2.83762032507,1
3,麵包,2.81338742394,2
3,餅乾,2.40307461567,3
4,肉乾,2.83762032507,1


### 個性化

In [44]:
# Build  recommender with personalization
personalized_model = tc.item_similarity_recommender.create(train_data ,user_id = 'member_id',
                                item_id='商品',target='qty',similarity_type='cosine')

Preparing data set.

Data has 66834 observations with 64191 users and 16 items.

Data prepared in: 0.150316s

Training model from provided data.

Gathering per-item and per-user statistics.

+--------------------------------+------------+

| Elapsed Time (Item Statistics) | % Complete |

In [45]:
# result = tc.recommender.util.compare_models(test_data, 
#                      [popularity_model, personalized_model],user_sample=0.5, skip_set=train_data)

+--------------------------------+------------+

| 815us                          | 1.5        |

| 7.875ms                        | 100        |

+--------------------------------+------------+

Setting up lookup tables.

...

| 14.168ms                            | 100              | 16              |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.025224s

In [46]:
# Applying the personalized model to make song recommendations
users = al2['member_id'].unique()  
print(len(al2['member_id'].unique()))

# personalized_model.recommend(users=[users[0]])
personalized_model.recommend(users = [users[6666]],k=2)

64503


member_id,商品,score,rank
57045,素食小吃,0.0549759864807,1
57045,雞肉,0.0451743602753,2


In [47]:
personalized_model.recommend(users = [users[6724]],k=5)

member_id,商品,score,rank
44010,素食小吃,0.164927959442,1
44010,雞肉,0.135523080826,2
44010,水果,0.0533623695374,3
44010,蝦子,0.0486159324646,4
44010,蛋捲,0.0457298755646,5


In [48]:
al2 = pd.DataFrame(al2)
al2.loc[al2.member_id == 70546]

,member_id,qty,商品
49941,70546,1,水果


In [49]:
#推薦商品 合起來
recom = personalized_model.recommend(users = users,k = 1)

df_rec1 = recom.to_dataframe()

df_rec1['recommendedProducts'] = df_rec1.groupby(['member_id'])['商品'] \
            .transform(lambda x: ','.join(x.astype(str)))

df_rec1 = df_rec1[['member_id', 'recommendedProducts']].drop_duplicates().sort_values('member_id').reset_index()

df_rec1 = df_rec1.drop(columns=['index'])

recommendations finished on 1000/64503 queries. users per second: 88370.4

recommendations finished on 2000/64503 queries. users per second: 163733

recommendations finished on 3000/64503 queries. users per second: 228120

recommendations finished on 4000/64503 queries. users per second: 284657

recommendations finished on 5000/64503 queries. users per second: 334986

...

recommendations finished on 60000/64503 queries. users per second: 792896

recommendations finished on 61000/64503 queries. users per second: 795908

recommendations finished on 62000/64503 queries. users per second: 798496

recommendations finished on 63000/64503 queries. users per second: 801792

recommendations finished on 64000/64503 queries. users per second: 802568

In [50]:
df_rec1 = pd.merge(df_rec1 , old_most, on ='member_id' ,how = 'inner' )
df_rec1['recommendedProducts'] = df_rec1['recommendedProducts'].str.cat(df_rec1['商品'],sep=',')
df_rec1 = df_rec1.drop(['qty','商品'],axis = 1)
df_rec1.tail()

,member_id,recommendedProducts
64498,86937,"小吃,補品"
64499,86938,"小吃,補品"
64500,86939,"蛋捲,餅乾"
64501,86940,"蛋捲,餅乾"
64502,86941,"素食小吃,小吃"
